In [6]:
from tkinter import *
import pandas as pd
import sqlite3
from tkinter.scrolledtext import ScrolledText
import logging
logging.basicConfig(filename='tmp.log',format='%(levelname)s %(asctime)s :: %(message)s',level=logging.DEBUG)
import sys
import pathlib

win = Tk()
win.title("EXCEL TO SQL CONVERTOR")
win.geometry("850x600")
win.configure(bg="lightblue")

txt1 = StringVar()
txt2 = StringVar()
txt3 = StringVar()
txt4 = StringVar()

e = Entry(win, width = 80, borderwidth = 5,textvariable = txt1)   
e.grid(row = 0, column = 1, columnspan = 3 , padx = 100, pady = 10)
f = Entry(win, width = 80, borderwidth = 5,textvariable = txt2)   
f.grid(row = 1, column = 1, columnspan = 3, padx = 100, pady = 10)
g = Entry(win, width = 80, borderwidth = 5,textvariable = txt3)   
g.grid(row = 4, column = 1, columnspan = 3 , padx = 100, pady = 10)
h = Entry(win, width = 80, borderwidth = 5,textvariable = txt4)   
h.grid(row = 5, column = 1, columnspan = 3, padx = 100, pady = 10)


t1 = ScrolledText(win, width = 80, height = 15, bg = "white")
t1.grid(row = 7, column = 0, columnspan = 3, padx = 80, pady = 10)

l1 = Label(win, text = 'NAME OF EXCEL FILE',anchor=W)
l1.grid(row = 0, column = 0, columnspan = 1)
l2 = Label(win, text = 'SHEET NAME OR NO.')
l2.grid(row = 1, column = 0, columnspan = 1)

l3 = Label(win, text = 'DATABASE NAME',anchor=W)
l3.grid(row = 4, column = 0, columnspan = 1)
l4 = Label(win, text = 'TABLE NAME')
l4.grid(row = 5, column = 0, columnspan = 1)
page = 0

def button_searchtables():
    logging.debug("Searching Table Names")
    filename=txt3.get()
    wb=pd.ExcelFile(filename+'.xlsx')
    ab=wb.sheet_names
    t1.insert(END,'\n')
    t1.insert(END,ab)
    
def button_export():
    logging.debug("Exporting")
    filename=txt1.get()
    sheet_names=txt2.get()
    z=0
    file = pathlib.Path(filename+'.xlsx')
    if file.exists ():
        con=sqlite3.connect(filename+".db")
        wb=pd.ExcelFile(filename+'.xlsx')
        for sheet in wb.sheet_names:
            df=pd.read_excel(filename+'.xlsx')
            df.to_sql(sheet_names,con, index=False,if_exists="replace")
            z=z+sys.getsizeof(df)
            t1.insert(END,"CONVERSION SUCCESSFUL...")
            t1.insert(END,"\nNO. OF BYTES CONVERTED:\t")
            t1.insert(END,z)
            t1.insert(END,"\n")
            f=pd.read_excel(filename+'.xlsx',index_col=0)
            a=f.dtypes
            t1.insert(END,a)
    else:
        t1.insert(END,"CONVERSION UNSUCCESSFUL..")
    con.commit()
    con.close()
    

def button_logs():
    with open('tmp.log') as f: 
        content = f.readlines()
        t1.insert(END,content)
    

def button_search():
    logging.debug("searching for database")
    name = txt3.get()
    cnx = sqlite3.connect(name+'.db')
    curs = cnx.cursor()
    cmd =  'select * from ' + txt4.get()    
    all_rec = curs.execute(cmd)
    one_rec = curs.fetchall()
    for row in one_rec:
        logging.debug("inserting")
        t1.insert(END,'\n')
        t1.insert(END,row)
        t1.insert(END,'\n')
    

    
    

button_searchtables = Button(win, text = "SEARCH TABLE", padx = 60, pady = 20,bg="black",fg="white", command = button_searchtables)
button_export = Button(win, text = "EXPORT TO DATABASE", padx = 60, pady = 20,bg="black",fg="white", command = button_export)
button_search = Button(win, text = "SEARCH FOR DATABASE", padx = 60, pady = 20,bg="black",fg="white", command = button_search)
button_logs = Button(win, text = "logs", padx = 60, pady = 20,bg="black",fg="white", command = button_logs)

button_searchtables.grid(row = 6, column = 0)
button_export.grid(row = 3, column = 1)
button_search.grid(row = 6, column = 1)
button_logs.grid(row = 6, column = 2)


win.mainloop()